Reformat and do 5-day averaging on daily sea ice data.

In [600]:
!wget -qN ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02135/north/daily/data/NH_seaice_extent_final.csv
!wget -qN ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02135/north/daily/data/NH_seaice_extent_nrt.csv
!wget -qN ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02135/south/daily/data/SH_seaice_extent_final.csv
!wget -qN ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02135/south/daily/data/SH_seaice_extent_nrt.csv


In [601]:
import datetime as dt
import numpy as np

import os
import pandas as pd
from pandas import ExcelWriter
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
pd.options.display.mpl_style = 'default'



In [602]:

def parse_the_date(year, mm, dd):
    return dt.date(int(year), int(mm), int(dd))

def slurp_csv(filename):
    data = pd.read_csv(filename, header = None, skiprows=2,
                       names=["year", "mm", "dd", "extent", "missing", "source"],
                       parse_dates={'date':['year', 'mm', 'dd']},
                       date_parser=parse_the_date, index_col='date')
    data = data.drop(['missing', 'source'], axis=1)
    return data


def read_a_hemisphere(hemisphere):
    final_prod_filename = os.path.join('{hemi}H_seaice_extent_final.csv'.format(hemi=hemisphere[0:1].upper()))
    nrt_prod_filename = os.path.join('{hemi}H_seaice_extent_nrt.csv'.format(hemi=hemisphere[0:1].upper()))

    final = slurp_csv(final_prod_filename)
    nrt = slurp_csv(nrt_prod_filename)
    all_data = pd.concat([final, nrt])
    return all_data



In [603]:
north = read_a_hemisphere('north')
south = read_a_hemisphere('south')
south.head()

            extent
date              
1978-10-26  17.634
1978-10-28  17.815
1978-10-30  17.671
1978-11-01  17.534
1978-11-03  17.493

Set indices to datetime indexes

In [572]:
north.index = pd.to_datetime(north.index)
south.index = pd.to_datetime(south.index)

In [604]:
north  = north.reindex(index=pd.date_range('1978-10-25', dt.date.today().strftime('%Y-%m-%d')))
south  = south.reindex(index=pd.date_range('1978-10-25', dt.date.today().strftime('%Y-%m-%d')))

In [605]:
north.head()

            extent
1978-10-25     NaN
1978-10-26  10.231
1978-10-27     NaN
1978-10-28  10.420
1978-10-29     NaN

In [606]:
south['hemi'] = 'South'
north['hemi'] = 'North'

In [648]:
df = north.copy()

## interpolate missing data in SMMR period.

We don't want to interpolate across any timeperiods where more than one day
of data is missing.  So we are going to union the NaN that remain after a
back fill and forward fill in order to leave any gaps in the data record
alone.

So start by using the backfill to fill any NaN locations that have a valid "next" value.
So start by using the forwardfill to fill any NaN locations that have a valid "previous" value.

In [649]:
df['backfill'] = df.extent.fillna(method='bfill', limit=1)
df['forwardfill'] = df.extent.fillna(method='ffill', limit=1)


In [650]:
df.head()

            extent   hemi  backfill  forwardfill
1978-10-25     NaN  North    10.231          NaN
1978-10-26  10.231  North    10.231       10.231
1978-10-27     NaN  North    10.420       10.231
1978-10-28  10.420  North    10.420       10.420
1978-10-29     NaN  North    10.557       10.420

In [651]:
df['19871201':'19871206']

            extent   hemi  backfill  forwardfill
1987-12-01  12.504  North    12.504       12.504
1987-12-02  12.600  North    12.600       12.600
1987-12-03     NaN  North       NaN       12.600
1987-12-04     NaN  North       NaN          NaN
1987-12-05     NaN  North       NaN          NaN
1987-12-06     NaN  North       NaN          NaN

In [652]:
df['19880110':'19880114']

            extent   hemi  backfill  forwardfill
1988-01-10     NaN  North       NaN          NaN
1988-01-11     NaN  North       NaN          NaN
1988-01-12     NaN  North    14.826          NaN
1988-01-13  14.826  North    14.826       14.826
1988-01-14  14.854  North    14.854       14.854

So the union of backfill's NaN and forward fill NaN will capture any missing
data that doesn't have a valid data point both previously and nextly in the series.

In [653]:
is_really_nan = pd.isnull(df['backfill']) | pd.isnull(df['forwardfill'])

Use the interpolation scheme to do simple linear regression

In [654]:
df['interpolated'] = df.extent.interpolate()

Mark missing any large gaps in the linearly interpolated data and drop the backfill and forwardfill columns

In [655]:
df[is_really_nan].interpolated = np.nan
df.drop(['forwardfill', 'backfill'], axis=1, inplace=True)

So now we have a simple dataframe with daily extents and daily interpolated extents

In [656]:
df.head()

            extent   hemi  interpolated
1978-10-25     NaN  North           NaN
1978-10-26  10.231  North       10.2310
1978-10-27     NaN  North       10.3255
1978-10-28  10.420  North       10.4200
1978-10-29     NaN  North       10.4885

Compute climatological means by fetching just the data between your desired climatology.

In [659]:
climatology_years = (1981, 2010)
clim_data = df[(df.index.year >= climatology_years[0])&(df.index.year <= climatology_years[1] )].copy()

In [664]:
print clim_data.head(),"\n...\n" ,clim_data.tail()

            extent   hemi  interpolated
1981-01-01  14.288  North       14.2880
1981-01-02     NaN  North       14.3955
1981-01-03  14.503  North       14.5030
1981-01-04     NaN  North       14.4810
1981-01-05  14.459  North       14.4590 
...
            extent   hemi  interpolated
2010-12-27  12.358  North        12.358
2010-12-28  12.398  North        12.398
2010-12-29  12.457  North        12.457
2010-12-30  12.558  North        12.558
2010-12-31  12.670  North        12.670


In [675]:
print len(np.unique(clim_data.index.year))
print np.unique(clim_data.index.year)

30
[1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995
 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010]


grab the mean value of the interpolated extents for each month/day combination

In [684]:
clim_averages = clim_data.groupby([clim_data.index.month, clim_data.index.day]).mean()[['interpolated']]


**check yourself**:  You can see in the three panels below that the value we get by calling
`mean()` on the `groupby` result is the same as expected by averaging the day
and month data separately

In [689]:
clim_data[(clim_data.index.month == 1)&(clim_data.index.day == 1)]

            extent   hemi  interpolated
1981-01-01  14.288  North       14.2880
1982-01-01     NaN  North       14.3710
1983-01-01  14.257  North       14.2570
1984-01-01     NaN  North       14.0065
1985-01-01     NaN  North       13.9460
1986-01-01  14.036  North       14.0360
1987-01-01     NaN  North       14.1970
1988-01-01     NaN  North       14.1900
1989-01-01  14.261  North       14.2610
1990-01-01  14.319  North       14.3190
1991-01-01  13.634  North       13.6340
1992-01-01  14.069  North       14.0690
1993-01-01  14.039  North       14.0390
1994-01-01  14.094  North       14.0940
1995-01-01  14.144  North       14.1440
1996-01-01  13.804  North       13.8040
1997-01-01  13.657  North       13.6570
1998-01-01  14.025  North       14.0250
1999-01-01  13.823  North       13.8230
2000-01-01  13.442  North       13.4420
2001-01-01  13.479  North       13.4790
2002-01-01  13.590  North       13.5900
2003-01-01  13.647  North       13.6470
2004-01-01  13.502  North       13.5020


In [690]:
clim_data[(clim_data.index.month == 1)&(clim_data.index.day == 1)].mean()

extent          13.725600
interpolated    13.795017
dtype: float64

In [691]:
clim_averages.head(1)

     interpolated
1 1     13.795017

In [692]:
clim_averages = clim_averages.rename(columns={'interpolated': '1981-2010'})
clim_averages.head(1)

     1981-2010
1 1  13.795017

In [695]:
clim_averages.index

MultiIndex(levels=[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0, 1, 2, 3, 4, 5, 6, 7, 8, ...]])

Set the daily extent data into the correct format for display and for concatenating with the clim_averages

In [707]:
df.index

<class 'pandas.tseries.index.DatetimeIndex'>
[1978-10-25, ..., 2015-05-05]
Length: 13342, Freq: D, Timezone: None

In [708]:
df = df[['extent']].set_index([df.index.year, df.index.month, df.index.day]).unstack(0)


In [710]:
df.index

MultiIndex(levels=[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0, 1, 2, 3, 4, 5, 6, 7, 8, ...]])

In [ ]:
Remove extent level from columns

In [715]:
print df.columns
df.columns = df.columns.droplevel()
print df.columns

Int64Index([1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015], dtype='int64')


In [750]:
a = clim_averages.copy()

In [754]:
a['1981-2010'] = "    "
a.rename(columns={'1981-2010': '   '}, inplace=True)


In [755]:
a

           
1  1       
   2       
   3       
   4       
   5       
   6       
   7       
   8       
   9       
   10      
   11      
   12      
   13      
   14      
   15      
   16      
   17      
   18      
   19      
   20      
   21      
   22      
   23      
   24      
   25      
   26      
   27      
   28      
   29      
   30      
...     ...
12 2       
   3       
   4       
   5       
   6       
   7       
   8       
   9       
   10      
   11      
   12      
   13      
   14      
   15      
   16      
   17      
   18      
   19      
   20      
   21      
   22      
   23      
   24      
   25      
   26      
   27      
   28      
   29      
   30      
   31      

[366 rows x 1 columns]

In [756]:
daily_extent_with_climatological_average = pd.concat([df,a, clim_averages], axis=1)
r = daily_extent_with_climatological_average

Add back a toplevel multi-index column

In [757]:
r.columns

Index([1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, u'   ', u'1981-2010'], dtype='object')

In [758]:
r['extent'] = 'Daily Extents with climatological means (based on interpolated days)'
r.set_index('extent', append=True, inplace=True)
r = r.unstack('extent')
r.columns =r.columns.reorder_levels(['extent', None])

In [759]:
import calendar
month_names = [calendar.month_name[x] for x in range(1,13)]



In [760]:
writer = ExcelWriter('../output/test_daily.xls')
r.to_excel(writer, float_format = "%.3f")
writer.save()

In [562]:
# cleanup
!rm -f NH_seaice_extent_final.csv NH_seaice_extent_nrt.csv SH_seaice_extent_final.csv SH_seaice_extent_nrt.csv
